In [2]:
import rioxarray
import numpy as np

In [3]:
xds = rioxarray.open_rasterio("./dems/larimerAreaDEM.tif")

In [4]:
xds_wgs = xds.rio.reproject("wgs84")

In [5]:
xds_wgs.rio.bounds()

(-106.00055555555541, 39.99944444471632, -104.99944444471632, 41.0005555555554)

In [56]:
xds_wgs[0].astype('int16')

<xarray.DataArray (y: 10812, x: 10812)>
array([[2402, 2403, 2403, ..., 1965, 1966, 1966],
       [2401, 2402, 2403, ..., 1966, 1966, 1967],
       [2400, 2403, 2403, ..., 1967, 1967, 1968],
       ...,
       [2795, 2796, 2798, ..., 1588, 1588, 1588],
       [2796, 2798, 2799, ..., 1588, 1588, 1588],
       [2798, 2798, 2799, ..., 1588, 1588, 1588]], dtype=int16)
Coordinates:
  * x            (x) float64 -106.0 -106.0 -106.0 ... -105.0 -105.0 -105.0
  * y            (y) float64 41.0 41.0 41.0 41.0 41.0 ... 40.0 40.0 40.0 40.0
    band         int64 1
    spatial_ref  int64 0
Attributes:
    LAYER_TYPE:                athematic
    RepresentationType:        ATHEMATIC
    STATISTICS_MAXIMUM:        4347.1943359375
    STATISTICS_MEAN:           2442.9052512532
    STATISTICS_MINIMUM:        1473.9787597656
    STATISTICS_STDDEV:         614.85092582736
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    long_name:                 Layer_1
    _FillValue:                -999999.0

In [6]:
elevMap = xds_wgs.isel()[0].values

In [7]:
xVals = xds_wgs.x.isel().values
xStart = xVals[0]
xStep = (xVals[-1] - xVals[0]) / (len(xVals)-1)

yVals = xds_wgs.y.isel().values
yStart = yVals[0]
yStep = (yVals[-1] - yVals[0]) / (len(yVals)-1)

In [8]:
e = elevMap.astype('int16')

In [9]:
e

array([[2402, 2403, 2403, ..., 1965, 1966, 1966],
       [2401, 2402, 2403, ..., 1966, 1966, 1967],
       [2400, 2403, 2403, ..., 1967, 1967, 1968],
       ...,
       [2795, 2796, 2798, ..., 1588, 1588, 1588],
       [2796, 2798, 2799, ..., 1588, 1588, 1588],
       [2798, 2798, 2799, ..., 1588, 1588, 1588]], dtype=int16)

In [10]:
geohashLength = 4

In [11]:
import geohash

In [12]:
geohash.bbox('9xj6')

{'s': 39.7265625, 'w': -105.1171875, 'n': 39.90234375, 'e': -104.765625}

In [17]:
print(xStart,yStart)
geohash.encode(yStart, xStart, 4)

-106.00050925925912 41.00050925925912


'9xk9'

In [32]:
def get_geohash_adjacent(gh, direction='e'):
    if not direction in 'nsew':
        raise Exception(f"{e} not in 'nsew'. This is not a valid direction")
    neighbors = geohash.neighbors(gh)
    m = {
        'w': 0,
        'e': 1,
        's': 2,
        'n': 5
    }
    return neighbors[m[direction]]

In [33]:
get_geohash_adjacent('9xh', 'e')

'9xj'

In [68]:
tlds = list(map(convert_tld_to_int16_wgs84_simple, [xds]))

In [95]:
def tld_bounds_and_geohash_bounds_intersect(tld_bounds, gh_bounds):
    print(tld_bounds)
    if tld_bounds[0] < gh_bounds['w'] < tld_bounds[2] or tld_bounds[0] < gh_bounds['e'] < tld_bounds[2] or tld_bounds[1] < gh_bounds['s'] < tld_bounds[3] or tld_bounds[1] < gh_bounds['n'] < tld_bounds[3]:
        return True
    return False

In [96]:
def convert_tld_to_int16_wgs84_simple(tld):
    tld_wgs = xds.rio.reproject("wgs84")
    return tld_wgs[0].astype('int16')

In [99]:
def CASE_1(tld, gh_bounds):
    pass

In [97]:
def build_dem_within_geohash(gh, tlds):
    gh_bounds = geohash.bbox(gh)
    print(gh_bounds)
    intersecting_tlds = list(filter(lambda tld: tld_bounds_and_geohash_bounds_intersect(tld.rio.bounds(), gh_bounds), tlds))
    if len(intersecting_tlds) == 1: #CASE 1
        

In [98]:
build_dem_within_geohash('9xhk', tlds)

{'s': 40.078125, 'w': -106.5234375, 'n': 40.25390625, 'e': -106.171875}
(-106.00055555555541, 39.99944444471632, -104.99944444471632, 41.0005555555554)
[<xarray.DataArray (y: 10812, x: 10812)>
array([[2402, 2403, 2403, ..., 1965, 1966, 1966],
       [2401, 2402, 2403, ..., 1966, 1966, 1967],
       [2400, 2403, 2403, ..., 1967, 1967, 1968],
       ...,
       [2795, 2796, 2798, ..., 1588, 1588, 1588],
       [2796, 2798, 2799, ..., 1588, 1588, 1588],
       [2798, 2798, 2799, ..., 1588, 1588, 1588]], dtype=int16)
Coordinates:
  * x            (x) float64 -106.0 -106.0 -106.0 ... -105.0 -105.0 -105.0
  * y            (y) float64 41.0 41.0 41.0 41.0 41.0 ... 40.0 40.0 40.0 40.0
    band         int64 1
    spatial_ref  int64 0
Attributes:
    LAYER_TYPE:                athematic
    RepresentationType:        ATHEMATIC
    STATISTICS_MAXIMUM:        4347.1943359375
    STATISTICS_MEAN:           2442.9052512532
    STATISTICS_MINIMUM:        1473.9787597656
    STATISTICS_STDDEV:        